In [1]:
# Imports
import numpy as np
import cv2
import pandas as pd
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import SGD
%matplotlib inline

Using TensorFlow backend.


In [2]:
# We define in advance constants to build the model

INPUT_SHAPE = (96, 96, 3)
OUTPUT_SIZE = 2

EPOCHS = 20
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 50

MODEL_PATH = "./net_0_model.h5"

In [3]:
# Use a random seed
random.seed(42)

In [4]:
# Load patches for sea lions
def extract_patches(path, size, label):
    data_set = []
    for file in range(0, size+1):
        patch = cv2.imread("./patches/" + path + "/" + str(file) + ".jpg")
        data_set.append(list((patch, label)))
    return data_set

In [5]:
# Extract patches
data_set_sea_lions_train = extract_patches("sealions_train", 62652, "sea lion")
data_set_sea_lions_test = extract_patches("sealions_test", 17634, "sea lion")
data_set_background_train = extract_patches("background_train", 62652, "background")
data_set_background_test = extract_patches("background_test", 17634, "background")

In [7]:
# Build train set
train_set = data_set_sea_lions_train + data_set_background_train
random.shuffle(train_set)
X_train = []
Y_train = []
for data in train_set:
    X_train.append(data[0])
    if data[1] == "sea lion":
        Y_train.append([1, 0])
    elif data[1] == "background":
        Y_train.append([0, 1])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Build test set
test_set = data_set_sea_lions_test + data_set_background_test
random.shuffle(test_set)
X_test = []
Y_test = []
for data in test_set:
    X_test.append(data[0])
    if data[1] == "sea lion":
        Y_test.append([1, 0])
    elif data[1] == "background":
        Y_test.append([0, 1])
X_test = np.array(X_test)
Y_test = np.array(Y_test)

# Convert data types and normalize values
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [8]:
# Build parallel model (multi gpu)

model = Sequential()
# First layer
model.add(Convolution2D(8, (5, 5), activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fourth layer
model.add(Convolution2D(10, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(OUTPUT_SIZE, activation='softmax'))

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# Train and test (multi gpu)

# Fit model on training data
parallel_model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, verbose=1)

# Evaluate model on test data
loss_and_metrics = parallel_model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=1)

# Save trained model (multi gpu)
parallel_model.save(MODEL_PATH)

Train on 8000 samples, validate on 2000 samples
Epoch 1/20
  50/8000 [..............................] - ETA: 7s - loss: 0.1923 - acc: 0.9000

 150/8000 [..............................] - ETA: 5s - loss: 0.2010 - acc: 0.8867

 250/8000 [..............................] - ETA: 5s - loss: 0.2251 - acc: 0.8960

 400/8000 [>.............................] - ETA: 4s - loss: 0.1864 - acc: 0.9225

 450/8000 [>.............................] - ETA: 5s - loss: 0.1957 - acc: 0.9200

...

3400/4000 [========================>.....] - ETA: 0s

3650/4000 [==========================>...] - ETA: 0s

3800/4000 [===========================>..] - ETA: 0s

4000/4000 [==============================] - 1s 267us/step


AttributeError: 'NoneType' object has no attribute 'update'

In [14]:
# Evaluate ROC and AUC

Y_pred = parallel_model.predict_proba(X_test)

roc_auc_score(Y_test, Y_pred)

AttributeError: 'Model' object has no attribute 'predict_proba'